In [1]:
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil

In [2]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Nov 15 09:32:26 AM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Nov 15 09:32:26 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 15 09:32:26 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 15 09:32:26 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 15 09:32:26 AM: Your problem is compiled with the CPP canonicalization backend.
----------------------------------------

(CVXPY) Nov 15 09:32:29 AM:   Dual.    obj: 4.2045433929e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 2e-08    cones: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Nov 15 09:32:29 AM: Problem status: optimal
(CVXPY) Nov 15 09:32:29 AM: Optimal value: 4.205e+01
(CVXPY) Nov 15 09:32:29 AM: Compilation took 1.058e+00 seconds
(CVXPY) Nov 15 09:32:29 AM: Solver (including time spent in interface) took 1.803e+00 seconds
Creating BEV..
Loading full influence matrix..
Done


In [3]:
# چون نمی توان تعداد سطرهای ماتریس اثر را کم کرد، و هدف کم کردن تعداد سطرهاست
#لذا برای اجرای الگوریتم نمونه برداری ستونی، ماتریس را ترانهاده می کنیم

t_matrix = A_full.transpose()

t_matrix.shape

(542, 60183)

<br><center> ***Minimize*** $\lVert BAx - Bd\rVert_{2}^2$

<br><center> ***Minimize*** $\lVert BA^Tx - Bd\rVert_{2}^2$

<br><center> ***Minimize*** $\lVert ABx - dB\rVert_{2}^2$
    
<br><center> ***Minimize*** $\lVert B^TAx - B^Td\rVert_{2}^2$
 
#کدام یک از این حالت ها در میاد؟  

In [8]:
from scipy.sparse import csr_matrix

def compute_probability(row):
    row_sum = row.sum()
    if row_sum == 0:
        return 0
    else:
        return row.sum() / row_sum

def insert_and_rescale(B, i, j, pi):
    B[:, j] += pi * t_matrix[:, i]

def compute_sparse_B(A_full, c):
    d = t_matrix.shape[1]
    if c < 1 or c > n:
        return csr_matrix((t_matrix.shape[1], c))

    num_rows, num_cols = A_full.shape
    B = csr_matrix((num_rows, c))   

    for i in range(d):
        pi = compute_probability(A_full[:, i])
        for j in range(c):
            insert_and_rescale(B, i, j, pi)

    return B


In [9]:
start_time = time.time()
n,d = A_full.shape
c = d 
result_B = compute_sparse_B(A_full, c)


end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان ساخت ماتریس : {total_execution_time} ثانیه")


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (542,)  and requested shape (60183,1)

In [10]:
c = n
result_B = compute_sparse_B(A_full, c)

process = psutil.Process()
memory_usage = process.memory_info().rss / 1024 / 1024

print(f"میزان حافظه مصرفی: {memory_usage:.2f} مگابایت")


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (542,)  and requested shape (60183,1)

In [12]:
result_B

<60183x60183 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [30]:
#c = 542   # با مقادیر بیشتر ارور میده
min_c = 1
max_c = A_sparse.shape[1] 
c = random.randint(min_c, max_c)
B = compute_sparse_B(A_full, c)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (542,)  and requested shape (60183,1)

In [14]:
B

<60183x60183 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [15]:
B.shape

(60183, 60183)

In [16]:
num_non_zero_elements_A = A_sparse.getnnz()  # تعداد عناصر غیر صفر
num_non_zero_elements_A

335573

In [17]:
num_non_zero_elements_B = B.getnnz()  # تعداد عناصر غیر صفر
num_non_zero_elements_B

0

In [25]:
sol_ls_B_sparse = scipy.sparse.linalg.lsqr(B, dose_sparse_1d)

In [19]:
sol_ls_B_sparse = scipy.sparse.linalg.lsqr(B, dose_sparse_1d)

# میزان حافظه مصرفی توسط متغیر sol_ls_B_sparse
memory_usage = sys.getsizeof(sol_ls_B_sparse)
print(f"میزان حافظه مصرفی: {memory_usage} بایت")


میزان حافظه مصرفی: 120 بایت


In [20]:
def my_function():
    sol_ls_B_sparse = scipy.sparse.linalg.lsqr(B, dose_sparse_1d)

cProfile.run("my_function()", sort="cumulative")


         319 function calls in 0.002 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.002    0.002 1974681953.py:1(my_function)
        1    0.000    0.000    0.002    0.002 lsqr.py:96(lsqr)
        1    0.000    0.000    0.001    0.001 _interface.py:250(rmatvec)
        1    0.000    0.000    0.001    0.001 _interface.py:297(_rmatvec)
        1    0.000    0.000    0.001    0.001 _interface.py:531(adjoint)
        1    0.000    0.000    0.001    0.001 _interface.py:800(_adjoint)
        1    0.000    0.000    0.001    0.001 _interface.py:806(__init__)
        2    0.000    0.000    0.000    0.000 _compressed.py:25(__init__)
       13    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       